In [ ]:
import pandas as pd
import numpy as np

from re import sub


# Car Prediction

Explicación variables
blablabla



##Importación de datos

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/JotaLeutgeb/proyectofinalDS1/main/Car_Prediction/car_price_prediction.csv')


## Entendiendo los datos


In [ ]:
df.head()

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


Nombres, cantidad y tipos de valores en columnas.


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19237 entries, 0 to 19236
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                19237 non-null  int64  
 1   Price             19237 non-null  int64  
 2   Levy              19237 non-null  object 
 3   Manufacturer      19237 non-null  object 
 4   Model             19237 non-null  object 
 5   Prod. year        19237 non-null  int64  
 6   Category          19237 non-null  object 
 7   Leather interior  19237 non-null  object 
 8   Fuel type         19237 non-null  object 
 9   Engine volume     19237 non-null  object 
 10  Mileage           19237 non-null  object 
 11  Cylinders         19237 non-null  float64
 12  Gear box type     19237 non-null  object 
 13  Drive wheels      19237 non-null  object 
 14  Doors             19237 non-null  object 
 15  Wheel             19237 non-null  object 
 16  Color             19237 non-null  object

In [ ]:
df.isna().sum()

ID                  0
Price               0
Levy                0
Manufacturer        0
Model               0
Prod. year          0
Category            0
Leather interior    0
Fuel type           0
Engine volume       0
Mileage             0
Cylinders           0
Gear box type       0
Drive wheels        0
Doors               0
Wheel               0
Color               0
Airbags             0
dtype: int64

Ninguna fila con datos nulos.

In [ ]:
for col in df.columns:
    print(f"columns  {col} : {df[col].nunique()}")
    print("****"*9)

columns  ID : 18924
************************************
columns  Price : 2315
************************************
columns  Levy : 559
************************************
columns  Manufacturer : 65
************************************
columns  Model : 1590
************************************
columns  Prod. year : 54
************************************
columns  Category : 11
************************************
columns  Leather interior : 2
************************************
columns  Fuel type : 7
************************************
columns  Engine volume : 107
************************************
columns  Mileage : 7687
************************************
columns  Cylinders : 13
************************************
columns  Gear box type : 4
************************************
columns  Drive wheels : 3
************************************
columns  Doors : 3
************************************
columns  Wheel : 2
************************************
columns  Color : 16
********

In [ ]:
df['Gear box type'].unique()

array(['Automatic', 'Tiptronic', 'Variator', 'Manual'], dtype=object)

## Análisis exploratorio y Limpieza de datos

In [ ]:
df.describe()

,ID,Price,Prod. year,Cylinders,Airbags
count,1.923700e+04,1.923700e+04,19237.000000,19237.000000,19237.000000
mean,4.557654e+07,1.855593e+04,2010.912824,4.582991,6.582627
std,9.365914e+05,1.905813e+05,5.668673,1.199933,4.320168
min,2.074688e+07,1.000000e+00,1939.000000,1.000000,0.000000
25%,4.569837e+07,5.331000e+03,2009.000000,4.000000,4.000000
50%,4.577231e+07,1.317200e+04,2012.000000,4.000000,6.000000
75%,4.580204e+07,2.207500e+04,2015.000000,4.000000,12.000000
max,4.581665e+07,2.630750e+07,2020.000000,16.000000,16.000000



###Organización y cambios preliminares en columnas

In [ ]:
df.rename(columns={'Prod. year': 'Prod year'}, inplace=True)

In [ ]:
df.columns = [col.replace(' ', '_') for col in df.columns]

In [ ]:
df.columns.tolist()

['ID',
 'Price',
 'Levy',
 'Manufacturer',
 'Model',
 'Prod_year',
 'Category',
 'Leather_interior',
 'Fuel_type',
 'Engine_volume',
 'Mileage',
 'Cylinders',
 'Gear_box_type',
 'Drive_wheels',
 'Doors',
 'Wheel',
 'Color',
 'Airbags']

***Columnas normalizadas y organizadas ✅***

##Limpieza

###Limpieza general del *DataFrame*

In [ ]:
df.duplicated().sum()

313

In [ ]:
df.drop_duplicates(inplace=True)

Filas duplicadas eliminadas ✅

In [ ]:
df=df.drop(['ID'],axis=1)

Eliminación de la columna `'ID'`, ya que tenemos una por defecto

### Limpieza `'Levy'`

In [ ]:
df['Levy'].value_counts()

-       5709
765      482
891      453
639      403
640      398
        ... 
3156       1
2908       1
1279       1
1719       1
1901       1
Name: Levy, Length: 559, dtype: int64

In [ ]:
df['Levy']=df['Levy'].replace('-','0')
df['Levy'].astype(float)

0        1399.0
1        1018.0
2           0.0
3         862.0
4         446.0
          ...  
19232       0.0
19233     831.0
19234     836.0
19235    1288.0
19236     753.0
Name: Levy, Length: 18924, dtype: float64

Reemplazo de "-" por 0 en `'Levy'` para tener solo float en la columna.

***Limpieza `'Levy'` ✅***

###Limpieza `'Mileage'`

Eliminación del str 'km' en la columna 'Mileage' para tener únicamente valores de tipo int.

In [ ]:
df['Mileage']=df['Mileage'].str.replace('km',"")

In [ ]:
df['Mileage'] =df['Mileage'].astype('int64')

***Limpieza `Mileage`✅***



###Limpieza `'Engine_volume'`

Borrando "turbo" para tener valores int en la columna 'Engine_volume"

In [ ]:
df['Engine_volume']=df['Engine_volume'].str.replace('Turbo',"")

In [ ]:
df['Engine_volume'] = df['Engine_volume'].astype('float64')

In [ ]:
df['Engine_volume'].sample(10)

14751    2.0
17516    3.5
14820    1.6
15854    3.5
8807     1.3
5757     1.8
8969     1.5
13687    4.6
6553     3.8
7970     4.0
Name: Engine_volume, dtype: float64

***Limpieza `Engine_Volume` ✅***

### Limpieza `'Airbags'`

Se encontró un error en los airbags, hay veces en la que indica 0 y eso no necesariamente está bien ya que los airbgas fueron reglamentarios a partir de 1998.

---

Aquellos valores que sean 0 a partir de ese año(Fecha en la que se volvieron obligatorios en USA), se reemplazarán por 2, que es el mínimo: (conductor y pasajero).

---

Probablemente queden airbags == 0, pero no serán valores erróneos sino correctos.

In [ ]:
df.loc[(df['Airbags'] == 0) & (df['Prod_year'] >= 1998), 'Airbags'] = 2

In [ ]:
df[(df['Airbags'] == 0) & (df['Prod_year'] >= 1998)]['Airbags'].count()

0

***Limpieza `'Airbags'` ✅***

### Limpieza `'Doors'`

In [ ]:
df['Doors'].unique()

array(['04-May', '02-Mar', '>5'], dtype=object)

Como se puede ver, la columna doors tiene 1 valor numérico, y seguido un mes.

Para limpiar esto, voy a utilizar la función `sub` de la librería `re`para dejar solo los valores numéricos

In [ ]:
def reemplazar_doors(valor):
  # Hay solo 3 valores en el dataframe: ['04-May', '02-Mar', '>5']
    if valor == '04-May':
        return '4-5'
    elif valor == '02-Mar':
        return '2-3'
    else:
        return valor # Si no aplican los condicionales anteriores, se mantiene el '>5'

In [ ]:
df['Doors'] = df['Doors'].astype(str)
df['Doors'] = df['Doors'].apply(reemplazar_doors)
df['Doors']

0        4-5
1        4-5
2        4-5
3        4-5
4        4-5
        ... 
19232    2-3
19233    4-5
19234    4-5
19235    4-5
19236    4-5
Name: Doors, Length: 18924, dtype: object

In [ ]:
df['Doors'].unique()

array(['4-5', '2-3', '>5'], dtype=object)

***Limpieza `'Doors'`✅***

###Limpieza `'Price'`

Analizando el precio se pueden ver valores menores a 1000, que parece un número demasiado bajo para el análisis.

In [ ]:
df[df['Price'] < 500]

,Price,Levy,Manufacturer,Model,Prod_year,Category,Leather_interior,Fuel_type,Engine_volume,Mileage,Cylinders,Gear_box_type,Drive_wheels,Doors,Wheel,Color,Airbags
27,30,0,TOYOTA,Prius,2008,Sedan,No,Hybrid,1.5,169000,4.0,Automatic,Front,4-5,Left wheel,White,8
33,125,1750,TOYOTA,Highlander,2008,Sedan,Yes,Hybrid,3.3,287274,6.0,Automatic,4x4,4-5,Left wheel,White,12
39,314,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,382746,6.0,Automatic,4x4,4-5,Left wheel,Silver,12
45,220,1194,TOYOTA,Tacoma,2013,Jeep,Yes,Petrol,4.0,239342,6.0,Automatic,4x4,4-5,Left wheel,Silver,12
53,392,1017,MERCEDES-BENZ,E 300,2017,Sedan,Yes,Petrol,2.0,1600,4.0,Automatic,Rear,4-5,Left wheel,Black,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19215,470,831,HYUNDAI,Elantra,2017,Sedan,Yes,Petrol,1.6,25635,4.0,Automatic,Front,4-5,Left wheel,Black,12
19218,30,0,NISSAN,Note,2006,Hatchback,No,Petrol,1.3,0,4.0,Automatic,Front,4-5,Right-hand drive,Green,9
19229,50,0,TOYOTA,Prius,2008,Hatchback,No,Hybrid,1.5,150000,4.0,Automatic,Front,4-5,Left wheel,Silver,6
19230,470,645,TOYOTA,Prius,2011,Hatchback,Yes,Hybrid,1.8,307325,4.0,Automatic,Front,4-5,Left wheel,Silver,12


Eliminamos todos los valores <= 1000 ya se consideraron como valores erróneos

In [ ]:
#df = df.drop(df[df['Price'] < 500].index)
df = df[df['Price'] >= 500]

In [ ]:
df[df['Price'] < 500]['Price'].count()


0

Filas con precio <= 1000 eliminadas

***Limpieza `'Price'`✅***

## Visualizaciones

In [ ]:
#aa

###Análisis de datos